# Keras Exercise

## Predict political party based on votes

1984년 미국 의회 의원이 17 가지 이슈에 투표한 공개 데이터셋.
투표 내역을 통해 소속 정당 ("Democrat" and "Republican") 분류.

In [1]:
import pandas as pd

import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [2]:
feature_names =  ['party','handicapped-infants', 'water-project-cost-sharing', 
                    'adoption-of-the-budget-resolution', 'physician-fee-freeze',
                    'el-salvador-aid', 'religious-groups-in-schools',
                    'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
                    'mx-missle', 'immigration', 'synfuels-corporation-cutback',
                    'education-spending', 'superfund-right-to-sue', 'crime',
                    'duty-free-exports', 'export-administration-act-south-africa']

voting_data = pd.read_csv('data/house-votes-84.data.txt', na_values=['?'], names = feature_names)
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


We can use describe() to get a feel of how the data looks in aggregate:

In [3]:
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


We can see there's some missing data to deal with here; some politicians abstained on some votes, or just weren't present when the vote was taken. We will just drop the rows with missing data to keep it simple, but in practice you'd want to first make sure that doing so didn't introduce any sort of bias into your analysis (if one party abstains more than another, that could be problematic for example.)

In [4]:
voting_data.dropna(inplace=True)
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


Our neural network needs normalized numbers, not strings, to work. So let's replace all the y's and n's with 1's and 0's, and represent the parties as 1's and 0's as well.

In [5]:
voting_data.replace(('y', 'n'), (1, 0), inplace=True)
voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)

In [6]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


Finally let's extract the features and labels in the form that Keras will expect:

In [7]:
all_features = voting_data[feature_names].drop('party', axis=1).values
all_classes = voting_data['party'].values

use cross_val_score to evaluate your resulting model with 10-fold cross-validation.

In [8]:
def create_model():
    model = Sequential()
    
    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    
    # Output layer with a binary classification (Democrat or Republican political party)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


# Wrap Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=1)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

Epoch 1/100
208/208 [==============================] - 0s 1ms/step - loss: 0.6925 - acc: 0.5048
Epoch 2/100
208/208 [==============================] - 0s 82us/step - loss: 0.6907 - acc: 0.5240
Epoch 3/100
208/208 [==============================] - 0s 79us/step - loss: 0.6877 - acc: 0.5240
Epoch 4/100
208/208 [==============================] - 0s 101us/step - loss: 0.6818 - acc: 0.6010
Epoch 5/100
208/208 [==============================] - 0s 89us/step - loss: 0.6723 - acc: 0.7788
Epoch 6/100
208/208 [==============================] - 0s 96us/step - loss: 0.6558 - acc: 0.7885
Epoch 7/100
208/208 [==============================] - 0s 103us/step - loss: 0.6331 - acc: 0.8125
Epoch 8/100
208/208 [==============================] - 0s 113us/step - loss: 0.6021 - acc: 0.8221
Epoch 9/100
208/208 [==============================] - 0s 110us/step - loss: 0.5651 - acc: 0.8269
Epoch 10/100
208/208 [==============================] - 0s 84us/step - loss: 0.5264 - acc: 0.8413
Epoch 11/100
208/208 [====

208/208 [==============================] - 0s 82us/step - loss: 0.0458 - acc: 0.9856
Epoch 85/100
208/208 [==============================] - 0s 62us/step - loss: 0.0453 - acc: 0.9904
Epoch 86/100
208/208 [==============================] - 0s 50us/step - loss: 0.0441 - acc: 0.9904
Epoch 87/100
208/208 [==============================] - 0s 50us/step - loss: 0.0436 - acc: 0.9904
Epoch 88/100
208/208 [==============================] - 0s 77us/step - loss: 0.0431 - acc: 0.9904
Epoch 89/100
208/208 [==============================] - 0s 62us/step - loss: 0.0444 - acc: 0.9904
Epoch 90/100
208/208 [==============================] - 0s 50us/step - loss: 0.0414 - acc: 0.9904
Epoch 91/100
208/208 [==============================] - 0s 48us/step - loss: 0.0423 - acc: 0.9808
Epoch 92/100
208/208 [==============================] - 0s 55us/step - loss: 0.0407 - acc: 0.9904
Epoch 93/100
208/208 [==============================] - 0s 50us/step - loss: 0.0404 - acc: 0.9904
Epoch 94/100
208/208 [===========

208/208 [==============================] - 0s 53us/step - loss: 0.0447 - acc: 0.9856
Epoch 68/100
208/208 [==============================] - 0s 50us/step - loss: 0.0433 - acc: 0.9856
Epoch 69/100
208/208 [==============================] - 0s 50us/step - loss: 0.0425 - acc: 0.9856
Epoch 70/100
208/208 [==============================] - 0s 72us/step - loss: 0.0423 - acc: 0.9856
Epoch 71/100
208/208 [==============================] - 0s 74us/step - loss: 0.0415 - acc: 0.9856
Epoch 72/100
208/208 [==============================] - 0s 53us/step - loss: 0.0412 - acc: 0.9904
Epoch 73/100
208/208 [==============================] - 0s 55us/step - loss: 0.0408 - acc: 0.9856
Epoch 74/100
208/208 [==============================] - 0s 55us/step - loss: 0.0394 - acc: 0.9856
Epoch 75/100
208/208 [==============================] - 0s 82us/step - loss: 0.0390 - acc: 0.9856
Epoch 76/100
208/208 [==============================] - 0s 77us/step - loss: 0.0384 - acc: 0.9856
Epoch 77/100
208/208 [===========

209/209 [==============================] - 0s 72us/step - loss: 0.0708 - acc: 0.9761
Epoch 51/100
209/209 [==============================] - 0s 53us/step - loss: 0.0695 - acc: 0.9809
Epoch 52/100
209/209 [==============================] - 0s 57us/step - loss: 0.0679 - acc: 0.9761
Epoch 53/100
209/209 [==============================] - 0s 74us/step - loss: 0.0666 - acc: 0.9761
Epoch 54/100
209/209 [==============================] - 0s 76us/step - loss: 0.0648 - acc: 0.9761
Epoch 55/100
209/209 [==============================] - 0s 57us/step - loss: 0.0643 - acc: 0.9809
Epoch 56/100
209/209 [==============================] - 0s 57us/step - loss: 0.0631 - acc: 0.9809
Epoch 57/100
209/209 [==============================] - 0s 50us/step - loss: 0.0617 - acc: 0.9856
Epoch 58/100
209/209 [==============================] - 0s 84us/step - loss: 0.0612 - acc: 0.9856
Epoch 59/100
209/209 [==============================] - 0s 76us/step - loss: 0.0602 - acc: 0.9856
Epoch 60/100
209/209 [===========

209/209 [==============================] - 0s 64us/step - loss: 0.0939 - acc: 0.9761
Epoch 34/100
209/209 [==============================] - 0s 55us/step - loss: 0.0906 - acc: 0.9809
Epoch 35/100
209/209 [==============================] - 0s 60us/step - loss: 0.0877 - acc: 0.9809
Epoch 36/100
209/209 [==============================] - 0s 105us/step - loss: 0.0850 - acc: 0.9809
Epoch 37/100
209/209 [==============================] - 0s 53us/step - loss: 0.0830 - acc: 0.9809
Epoch 38/100
209/209 [==============================] - 0s 60us/step - loss: 0.0807 - acc: 0.9809
Epoch 39/100
209/209 [==============================] - 0s 79us/step - loss: 0.0784 - acc: 0.9809
Epoch 40/100
209/209 [==============================] - 0s 107us/step - loss: 0.0775 - acc: 0.9809
Epoch 41/100
209/209 [==============================] - 0s 72us/step - loss: 0.0760 - acc: 0.9809
Epoch 42/100
209/209 [==============================] - 0s 110us/step - loss: 0.0757 - acc: 0.9761
Epoch 43/100
209/209 [========

...

209/209 [==============================] - 0s 67us/step - loss: 0.0621 - acc: 0.9856
Epoch 49/100
209/209 [==============================] - 0s 93us/step - loss: 0.0593 - acc: 0.9904
Epoch 50/100
209/209 [==============================] - 0s 64us/step - loss: 0.0570 - acc: 0.9856
Epoch 51/100
209/209 [==============================] - 0s 69us/step - loss: 0.0568 - acc: 0.9809
Epoch 52/100
209/209 [==============================] - 0s 96us/step - loss: 0.0543 - acc: 0.9856
Epoch 53/100
209/209 [==============================] - 0s 86us/step - loss: 0.0528 - acc: 0.9904
Epoch 54/100
209/209 [==============================] - 0s 93us/step - loss: 0.0521 - acc: 0.9904
Epoch 55/100
209/209 [==============================] - 0s 93us/step - loss: 0.0506 - acc: 0.9904
Epoch 56/100
209/209 [==============================] - 0s 96us/step - loss: 0.0491 - acc: 0.9904
Epoch 57/100
209/209 [==============================] - 0s 96us/step - loss: 0.0472 - acc: 0.9904
Epoch 58/100
209/209 [===========

209/209 [==============================] - 0s 60us/step - loss: 0.0910 - acc: 0.9713
Epoch 32/100
209/209 [==============================] - 0s 84us/step - loss: 0.0879 - acc: 0.9713
Epoch 33/100
209/209 [==============================] - 0s 79us/step - loss: 0.0842 - acc: 0.9713
Epoch 34/100
209/209 [==============================] - 0s 67us/step - loss: 0.0818 - acc: 0.9713
Epoch 35/100
209/209 [==============================] - 0s 69us/step - loss: 0.0782 - acc: 0.9713
Epoch 36/100
209/209 [==============================] - 0s 107us/step - loss: 0.0752 - acc: 0.9713
Epoch 37/100
209/209 [==============================] - 0s 86us/step - loss: 0.0743 - acc: 0.9713
Epoch 38/100
209/209 [==============================] - 0s 110us/step - loss: 0.0716 - acc: 0.9713
Epoch 39/100
209/209 [==============================] - 0s 100us/step - loss: 0.0689 - acc: 0.9713
Epoch 40/100
209/209 [==============================] - 0s 86us/step - loss: 0.0675 - acc: 0.9713
Epoch 41/100
209/209 [========

209/209 [==============================] - 0s 57us/step - loss: 0.4083 - acc: 0.8995
Epoch 14/100
209/209 [==============================] - 0s 69us/step - loss: 0.3627 - acc: 0.9043
Epoch 15/100
209/209 [==============================] - 0s 81us/step - loss: 0.3234 - acc: 0.8995
Epoch 16/100
209/209 [==============================] - 0s 69us/step - loss: 0.2923 - acc: 0.9043
Epoch 17/100
209/209 [==============================] - 0s 72us/step - loss: 0.2640 - acc: 0.9234
Epoch 18/100
209/209 [==============================] - 0s 88us/step - loss: 0.2441 - acc: 0.9234
Epoch 19/100
209/209 [==============================] - 0s 91us/step - loss: 0.2266 - acc: 0.9234
Epoch 20/100
209/209 [==============================] - 0s 91us/step - loss: 0.2139 - acc: 0.9234
Epoch 21/100
209/209 [==============================] - 0s 86us/step - loss: 0.2025 - acc: 0.9234
Epoch 22/100
209/209 [==============================] - 0s 100us/step - loss: 0.1929 - acc: 0.9282
Epoch 23/100
209/209 [==========

209/209 [==============================] - 0s 72us/step - loss: 0.0368 - acc: 0.9904
Epoch 97/100
209/209 [==============================] - 0s 81us/step - loss: 0.0359 - acc: 0.9904
Epoch 98/100
209/209 [==============================] - 0s 76us/step - loss: 0.0365 - acc: 0.9856
Epoch 99/100
209/209 [==============================] - 0s 62us/step - loss: 0.0351 - acc: 0.9904
Epoch 100/100
23/23 [==============================] - 0s 6ms/step


0.9438405811786652